In [4]:
!kill -9 32762

/bin/sh: 1: kill: No such process



In [16]:
!ls ../output

shalowunet  unified


In [5]:
import os
checkpoint = '../output/unified/2'
frozen_dir = checkpoint+'/frozen'
os.system('rm -r {}'.format(frozen_dir))
cmd = 'python ../unified_network.py --mode export \
    --checkpoint {} --crop_size 1024 --output_dir {} --ngf 32 --down_scale_rate 2'.format(checkpoint, frozen_dir)
print(cmd)
os.system(cmd)

python ../unified_network.py --mode export     --checkpoint ../output/unified/2 --crop_size 1024 --output_dir ../output/unified/2/frozen --ngf 32 --down_scale_rate 2


256

In [19]:
from time import time
import tensorflow as tf
import cv2
import numpy as np
import os
from glob import glob
import math


def get_tensor_by_name(name):
    name_on_device = '{}:0'.format(name)
    return tf.get_default_graph().get_tensor_by_name(name_on_device)

def load_image(path, verbal=False, down_scale=1):
    print('---------------------------\nprocess:', path)
    name = path.split('/')[-1].split('.')[0]
    image = cv2.imread(path)
#     p = 286/256
    image = cv2.resize(image, (0,0), fx=down_scale, fy=down_scale)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image[:, :image.shape[1]//2]

def write(path, image):
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return cv2.imwrite(path, image)
print('Finish')
def get_name(path):
    name, _ = os.path.splitext(os.path.basename(path))
    return name   

Finish


In [20]:
def save_output(image, output_image, save_dir):
    os.makedirs(save_dir, exist_ok=True)
    inp = image
    out1 = output_image[:,:,0]
    out2 = output_image[:,:,-1]
    #cv2.imwrite(os.path.join(save_dir, 'input.png'), inp)
    write(os.path.join(save_dir, 'input.png'), inp)
    cv2.imwrite(os.path.join(save_dir, 'output1.png'), out1)
    cv2.imwrite(os.path.join(save_dir, 'output2.png'), out2)
    print('Output: ', save_dir)

In [21]:
def run(sess, feed_image, batch_size=8):
    batch_input = sess.run(batch_input_tensor, {inputs:feed_image})
    rv = []
    
    for i in range(0, len(batch_input), batch_size):
        print('\r {:0.2f} %'.format(i/len(batch_input)), end='')
        rv.append(sess.run(batch_output_tensor, {batch_input_placeholder: batch_input[i:i+batch_size]}))
    output_feed = np.concatenate(rv, axis=0)
    return sess.run(outputs, {batch_output_placeholder: output_feed, inputs:feed_image})

# RESTORE MODEL


In [22]:
frozen_dir = '../output/shalowunet/dic/frozen/'

In [23]:
# input_dir = '../data/bprost/refine2/*/*'
checkpoint = frozen_dir
output_dir = os.path.join(checkpoint, 'images')

tf.reset_default_graph()
from time import time
meta_path = os.path.join(checkpoint, 'export.meta')
print('meta path:', meta_path)

assert os.path.exists(meta_path), meta_path+' does not exist'
tf.train.import_meta_graph(meta_path)
saver = tf.train.Saver()

meta path: ../output/shalowunet/dic/frozen/export.meta


# RUN SAMPLE

In [24]:
input_dir = '../data/dic/origin_combine/'

In [25]:
print('input_dir:', input_dir)
paths = glob('{}/*.png'.format(input_dir))
# paths = [path for path in paths if get_name(path).isdigit()]
print(paths)

input_dir: ../data/dic/origin_combine/
['../data/dic/origin_combine/SG1-1-2.png', '../data/dic/origin_combine/SG1-11.png', '../data/dic/origin_combine/SG1-29.png', '../data/dic/origin_combine/SG1-33-2.png', '../data/dic/origin_combine/SG1-40-2.png', '../data/dic/origin_combine/SG1-26.png', '../data/dic/origin_combine/SG1-30.png', '../data/dic/origin_combine/SG1-4.png', '../data/dic/origin_combine/SG1-20.png', '../data/dic/origin_combine/SG1-40-1.png']


In [26]:
down_scale = 286/256

In [27]:
with tf.Session() as sess:

    saver.restore(sess, tf.train.latest_checkpoint(
        checkpoint))

    parameter_count = tf.reduce_sum([tf.reduce_prod(tf.shape(v)) for v in tf.trainable_variables()])
    print('num of params:', sess.run(parameter_count))


    os.makedirs(output_dir, exist_ok=True)
    start = time()

    assert len(paths) > 0
    print('Num of sample:', len(paths), input_dir)
    inputs = get_tensor_by_name('inputs')
    outputs = get_tensor_by_name('outputs')
    batch_input_tensor = get_tensor_by_name('batch_input_tensor')
    batch_input_placeholder = get_tensor_by_name('batch_input_placeholder')
    batch_output_tensor = get_tensor_by_name('batch_output_tensor')
    batch_output_placeholder = get_tensor_by_name('batch_output_placeholder')
    start = time()
    begin = start
    for path in paths:
        name = path.split('/')[-2].split('.')[0]
        image = load_image(path, verbal=True, down_scale=down_scale)
        print('image:', path, '\t shape: ', image.shape)
        output_image = run(sess, image)
        save_dir = os.path.join(checkpoint, 'temp1/{}'.format(name))
        image = cv2.resize(image, (0,0), fx=1/down_scale, fy=1/down_scale)
        output_image = cv2.resize(output_image, (0,0), fx=1/down_scale, fy=1/down_scale)
        save_output(image, output_image, save_dir)
        print('Running time:', time()-start, 'image:', image.shape)


    print('Running time:', time()-begin)


INFO:tensorflow:Restoring parameters from ../output/shalowunet/dic/frozen/export
num of params: 282345
Num of sample: 10 ../data/dic/origin_combine/
---------------------------
process: ../data/dic/origin_combine/SG1-1-2.png
image: ../data/dic/origin_combine/SG1-1-2.png 	 shape:  (2603, 1852, 3)
 0.93 %Output:  ../output/shalowunet/dic/frozen/temp1/origin_combine
Running time: 2.2536795139312744 image: (2330, 1658, 3)
---------------------------
process: ../data/dic/origin_combine/SG1-11.png
image: ../data/dic/origin_combine/SG1-11.png 	 shape:  (1846, 1340, 3)
 0.89 %Output:  ../output/shalowunet/dic/frozen/temp1/origin_combine
Running time: 3.4080581665039062 image: (1652, 1199, 3)
---------------------------
process: ../data/dic/origin_combine/SG1-29.png
image: ../data/dic/origin_combine/SG1-29.png 	 shape:  (2608, 1852, 3)
 0.93 %Output:  ../output/shalowunet/dic/frozen/temp1/origin_combine
Running time: 5.395366430282593 image: (2334, 1658, 3)
---------------------------
process: 